In [ ]:
import matplotlib.pyplot as plt
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense
from keras.optimizers import Adam
import keras
from keras.callbacks import ModelCheckpoint
import os
import time
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
labels = os.listdir('../input/kermany2018/OCT2017 /train/')

train_datagen = ImageDataGenerator(samplewise_center=True, 
                              samplewise_std_normalization=True, 
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.1, 
                              width_shift_range=0.1, 
                              rotation_range=15, 
                              zoom_range=0.15)
validation_datagen = ImageDataGenerator(samplewise_center=True, 
                                        samplewise_std_normalization=True)
test_datagen = ImageDataGenerator(samplewise_center=True, 
                                  samplewise_std_normalization=True)

In [ ]:
train_data_dir = '../input/kermany2018/OCT2017 /train'
validation_data_dir = '../input/kermany2018/OCT2017 /val'
test_data_dir = '../input/kermany2018/OCT2017 /test'

In [ ]:
image_size = 224

valid_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(image_size ,image_size),
    batch_size= 32,
    class_mode='categorical')
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_size, image_size),
    batch_size= 42,
    class_mode='categorical',
    shuffle=True)

In [ ]:
# defining inceptionv3 model
t_x, t_y = next(train_generator)

def inception_v3():
    img_in = Input(t_x.shape[1:])              
    model =  InceptionV3(include_top= False , 
                weights='imagenet',      
                input_tensor= img_in, 
                input_shape= t_x.shape[1:],
                pooling ='avg') 
    x = model.output  
    predictions = Dense(4, activation="softmax", name="predictions")(x)    
    model = Model(inputs=img_in, outputs=predictions)
    return model

In [ ]:
# compiling model
model = inception_v3()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                           metrics = ['accuracy'])

In [ ]:
# creating checkpoint
file_path="weights.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# training model
history = model.fit_generator(train_generator, 
                                  steps_per_epoch=100,
                                  validation_data = valid_generator, 
                                  epochs = 20,
                                  callbacks=callbacks_list)

In [ ]:
# creating test data generator
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(image_size , image_size),
    batch_size=968,
    class_mode='categorical')
test_X,test_Y = next(test_generator)

In [ ]:
# loading the model with best weight
model = inception_v3()
model.load_weights(file_path)

In [ ]:
# predicting y value
start = time.time()
y_pred = model.predict(test_X)
end = time.time()

print('Test time per image:')
print((end - start)/968)

In [ ]:
Y_pred = []
for i in range(len(y_pred)):
    Y_pred.append(np.argmax(y_pred[i]))
    
Y_test = []
for i in range(len(test_Y)):
    Y_test.append(np.argmax(test_Y[i]))
    
print(classification_report(Y_test, Y_pred))

In [ ]:
# Creating a confusion matrix represented as a heat map
labels = [0,1,2,3]
sns.heatmap(confusion_matrix(Y_test, Y_pred), annot=True, fmt=".3f", xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')
plt.show()